In [8]:
from tars import Tars, markets, portfolios, traders, strategies
from tars.utils import data
import pandas as pd
import numpy as np
%matplotlib inline

# Backtester



    1. tars.test(data)         - load the data as a user
                               - start the strategies in different threads with HistoricalRunner
                                   
    
    2. HistoricalRunner(data)  - start : start the strategy run

                                    for each time in time index
                                        - execute strategy.test(data) i.e. the part of the code where the strategy is
                                            - look at the market with HistoricalMarket
                                            - take a decision with the strategy
                                            - place an order with HistoricalTrader
                                            
                                - stop : stop the execution
                                    



    HistoricalTrader(data)      - place orders in an order book at a given time stamp
                                - can place orders within the time frame of the data


    HistoricalMarket(data)      - can load OHLC data
                                - can give the same results as normal market object on the subset of data that has been loaded
              

## During strategy development

In [25]:
import pandas as pd
from tars.evaluators.trader_evaluator import TraderEvaluator
from tars.strategies.abstract_strategy import AbstractStrategy
from tars.markets.crypto_market import CryptoMarket
import random
import logging


class PredictionStrategy(AbstractStrategy):
    """
    Prediction Strategy

    :param trader: Trader
        The Trader handling a portfolio
    :param pair: str
        The pair e.g. XETHZUSD to buy and hold
    :param volume: float
        The volume of the pair's quote buy
    :param validate: boolean
        Safety Boolean to make sure not to trade real money by default

    :ivar evaluator: AbstractEvaluator
        Evaluator allows for the evaluation of a strategy
    :ivar market: AbstractMarket
        Market object to get information from
    """

    def __init__(self, trader, pair, volume, validate=True):
        
        # Strategy params
        self.name = 'Prediction'
        self.trader = trader
        self.pair = pair
        self.volume = volume
        
        # Generic params
        self.validate = validate
        self.evaluator = TraderEvaluator(self.trader)

    def run(self):
        """ Run the strategy """
        # 1. Add a checkpoint to the evaluator
        balance = self.trader.portfolio.get_trade_balance().loc['eb'].ZUSD   
        self.evaluator.add_checkpoint(pd.Timestamp.utcnow(), balance)
        
        # 2. Process the data
        market = CryptoMarket()

        # get last frame
        df = self.market.get_ohlc_data(pair=self.pair, ascending=True, interval=15)[0]

        # preprocessing
        ts = TimeSeries.from_dataframe(df.reset_index(), 'dtime', 'close')

        # modeling
        model = ExponentialSmoothing()
        model.fit(ts)

        # 3. Get the relevant informations for trading decision
        prediction = round(model.predict(2).last_value(),2)
        current = df.iloc[-1]['close']

        # 4. Implement the trading logic
        if prediction < current:
            self.trader.add_order(pair=self.pair, type='sell',
                                  ordertype='market', volume=self.volume,
                                  validate=self.validate)

        elif prediction >= current:
            self.trader.add_order(pair=self.pair, type='buy',
                                  ordertype='market', volume=self.volume,
                                  validate=self.validate)
        else:
            pass
        
    def test(self, dtime, data):
        """ Backtest the strategy """
        order = ['buy', 'sell'][random.getrandbits(1)]
        logging.info(data.loc[dtime])
        logging.info(f'Order type : {order}')
        

## During usage

In [26]:
api_key = f'../kraken.key'

In [27]:
portfolio_1 = portfolios.VirtualPortfolio({'ZUSD': 1000})
trader_1 = traders.VirtualCryptoTrader(portfolio_1)
strategy_1 = strategies.BuyAndHold(trader_1, 'XETHZUSD', 0.2)

portfolio_2 = portfolios.VirtualPortfolio({'ZUSD': 1000})
trader_2 = traders.VirtualCryptoTrader(portfolio_2)
strategy_2 = PredictionStrategy(trader_2, 'XETHZUSD', 0.2)

In [28]:
# Create Tars and load strategies
tars = Tars()
#tars.load(strategy_1)
tars.load(strategy_2)

 🤖 TARS : Welcome to Endurance! 👨‍🚀
 🤖 TARS : Loaded strategy ➡️ Prediction


### Run on a dataset to backtest

In [29]:
from tars.utils import data
my_data = data.get_historical_ohlc_data(base='ETH', quote='USD', interval=1, data_folder='../../data/raw/')['2021':]

In [30]:
my_data

,open,high,low,close,volume,trades
timestamp,,,,,,
2021-01-01 00:00:00,737.35,737.93,737.35,737.93,48.216264,12
2021-01-01 00:01:00,737.93,739.78,737.93,739.78,14.375177,19
2021-01-01 00:02:00,739.69,739.69,738.99,738.99,3.522163,6
2021-01-01 00:03:00,738.71,738.97,738.58,738.91,127.772880,16
2021-01-01 00:04:00,738.41,738.41,738.36,738.36,2.948720,4
...,...,...,...,...,...,...
2021-06-30 23:55:00,2273.17,2273.17,2271.51,2271.60,9.068894,11
2021-06-30 23:56:00,2271.59,2271.91,2271.59,2271.90,34.495100,21
2021-06-30 23:57:00,2271.91,2274.59,2271.91,2273.90,13.735273,12


In [31]:
tars.test(my_data.head())

 🤖 TARS : Starting backtesting session 📈
 💪️ Loading :   
   🧵 'Thread-10' ➡️ 'Prediction'
 open      737.350000
high      737.930000
low       737.350000
close     737.930000
volume     48.216264
trades     12.000000
Name: 2021-01-01 00:00:00, dtype: float64
 Order type : sell
 open      737.930000
high      739.780000
low       737.930000
close     739.780000
volume     14.375177
trades     19.000000
Name: 2021-01-01 00:01:00, dtype: float64
 Order type : sell
 open      739.690000
high      739.690000
low       738.990000
close     738.990000
volume      3.522163
trades      6.000000
Name: 2021-01-01 00:02:00, dtype: float64
 Order type : buy
 open      738.71000
high      738.97000
low       738.58000
close     738.91000
volume    127.77288
trades     16.00000
Name: 2021-01-01 00:03:00, dtype: float64
 Order type : sell
 open      738.41000
high      738.41000
low       738.36000
close     738.36000
volume      2.94872
trades      4.00000
Name: 2021-01-01 00:04:00, dtype: float64
 

In [23]:
---

tars.plot()

SyntaxError: invalid syntax (1783557540.py, line 1)

In [16]:
tars.evaluate()

,Buy and hold,Sequential Investment
2021-08-11 07:39:00+00:00,1000.00,1000.00
2021-08-11 07:40:00+00:00,997.37,998.04
2021-08-11 07:41:00+00:00,997.46,998.13


In [20]:
tars.stop()

---

In [ ]:
def backtest(data, frequency, duration=None) -> NoReturn:
        """
        Start a backtesting session
        :param data: DataFrame
            OHLCV Data in a Pandas DataFrame 
        :param frequency: str
            Frequency in the same form than a Pandas' Timedelta
        :param duration: str
            Frequency in the same form than a Pandas' Timedelta
        """
        if self.strategies:
            logging.info(f"{self._tars} : Starting backtesting session 📈")
            logging.info(f"⏱ Trading decision will be taken every : {frequency}️ (hh:mm:ss)")
            if duration is not None:
                logging.info(f"⏳ for a duration of : {duration} (hh:mm:ss)")
            logging.info(f"💪️ Loading :   ")
            for i, s in enumerate(self.strategies):
                runner = BacktestRunner()
                self.runners.append(runner)
                thread = Thread(target=runner.start,
                                args=(data, s.run, frequency, duration))
                thread.start()
                logging.info(f"  🧵 '{thread.name}' ➡️ '{s.name}'")
            self.is_running = True
        else:
            raise Exception('There are no loaded strategies')

In [ ]:
import logging
from typing import Callable, NoReturn
from time import sleep

from pandas import Timestamp, Timedelta


class BacktestRunner:
    """
    A BacktestRunner represent an object able to execute a function through time on a given dataset.

    The function can be executed with a chosen frequency e.g. every 10 seconds
    and for a optional duration e.g. 2 hours.

    :ivar is_running : Boolean describing if the Runner is running or not.
    """
      
    def __init__(self):
        self.is_running = False

    def start(self, data: DataFrame, func: Callable, frequency: str, duration: str = None) \
            -> NoReturn:
        """ Start the Runner

        :param data: The data to use
        :param func: The function to be executed
        :param frequency: String representing a frequency in the same form than a Pandas' Timedelta (https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html)
        :param duration: String representing a frequency in the same form than a Pandas' Timedelta (https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html)
        """
        self.is_running = True
        
        # Initialize
        start_time = curr_year.index[0]
        end_time = curr_year.index[-1]
        current_time = start

        if duration is not None:
            duration_end_time = start + Timedelta(duration)
        
        while self.is_running or current_time <= end_time:
            if duration is not None:
                if current_time >= duration_end_time:
                    break
            func()
            current_time += Timedelta(frequency)

        logging.debug(f'Runner started with frequency of {frequency} and '
                      f'duration of {duration}')

    def stop(self) -> NoReturn:
        """ Stop the Runner """
        self.is_running = False
        logging.debug(f'Runner stopped')
